# Some examples on numerical differentiation in Python

## Pure Python

* Python doesn’t *natively* support vectorization. 
* There are two reasons for this:
    - Python lists store pointers to the actual data,
    - Python bytecode is not optimized for vectorization, so for loops cannot predict when using vectorization would be beneficial.


In [ ]:
import time

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

### diffusion problem

In [ ]:
grid_shape = (1024, 1024)

In [ ]:
def evolve(grid, dt, out, D=1.0):
    xmax, ymax = grid_shape
    for i in range(xmax):
        for j in range(ymax):
            grid_xx = grid[(i+1)%xmax][j] + grid[(i-1)%xmax][j] - 2.0 * grid[i][j]
            grid_yy = grid[i][(j+1)%ymax] + grid[i][(j-1)%ymax] - 2.0 * grid[i][j]
            out[i][j] = grid[i][j] + D * (grid_xx + grid_yy) * dt

In [ ]:
def run_experiment(num_iterations):
# Setting up initial conditions
    xmax, ymax = grid_shape
    
    next_grid = [[0.0,] * ymax for x in range(xmax)]
    grid = [[0.0,] * ymax for x in range(xmax)]
    
    block_low = int(grid_shape[0] * .4)
    block_high = int(grid_shape[0] * .5)
    for i in range(block_low, block_high):
        for j in range(block_low, block_high):
            grid[i][j] = 0.005
    
    # Start of integration
    start = time.time()
    for i in range(num_iterations):
        evolve(grid, 0.1, next_grid)
        grid, next_grid = next_grid, grid
    return time.time() - start

In [ ]:
run_experiment(10)

## NumPy

* Luckily, numpy has all of the features we need - it stores data in **contiguous chunks of memory**
* It supports vectorized operations on its data
* As a result, any arithmetic we do on numpy arrays happens in chunks without us having to explicitly loop over each element

## Example to calculate vector norm

In [ ]:
import numpy as np

In [ ]:
vector = list(range(1_000_000))

#### pure python

In [ ]:
def norm_square_list(vector):
    norm = 0
    for v in vector:
        norm += v * v
    return norm

In [ ]:
%timeit norm_square_list(vector)

In [ ]:
def norm_square_list_comprehension(vector):
    return sum([v*v for v in vector])

In [ ]:
%timeit norm_square_list_comprehension(vector)

In [ ]:
def norm_squared_generator_comprehension(vector):
    return sum(v*v for v in vector)

In [ ]:
%timeit norm_squared_generator_comprehension(vector)

#### the same, but with numpy

In [ ]:
vector = np.arange(1_000_000)

In [ ]:
def norm_square_numpy(vector):
    return np.sum(vector * vector)

In [ ]:
%timeit norm_square_numpy(vector)

In [ ]:
def norm_square_numpy_dot(vector):
    return np.dot(vector, vector) 

In [ ]:
%timeit norm_square_numpy_dot(vector)

## Applying numpy to finite differences

Generic differences

* `numpy.gradient()`

* `numpy.diff()`

In [ ]:
a = np.arange(10, 20, 2)
a

In [ ]:
np.diff(a)

In [ ]:
a[1:] - a[:-1]

### Diffusion Problem

In [ ]:
grid_shape = (1024, 1024)

In [ ]:
def laplacian(grid):
    return (np.roll(grid, +1, 0) + np.roll(grid, -1, 0) +
            np.roll(grid, +1, 1) + np.roll(grid, -1, 1) - 4 * grid)

In [ ]:
def evolve(grid, dt, D=1):
    return grid + dt * D * laplacian(grid)

In [ ]:
def run_experiment_numpy(num_iterations):
    grid = np.zeros(grid_shape)
    block_low = int(grid_shape[0] * .4)
    block_high = int(grid_shape[0] * .5)
    grid[block_low:block_high, block_low:block_high] = 0.005
    grid0 = grid.copy()
    
    start = time.time()
    for i in range(num_iterations):
        grid = evolve(grid, 0.1)
    return grid0, grid

In [ ]:
n = 1000
g0, g = run_experiment_numpy(n)

In [ ]:
fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(16, 6))

h = ax0.pcolormesh(g0)
fig.colorbar(h, ax=ax0)
ax0.set_title('Initial conditions')

h = ax1.pcolormesh(g)
fig.colorbar(h, ax=ax1)
ax1.set_title('After {n} iterations'.format(n=n))